In [ ]:
# Modified from an R script found on Kaggle - https://www.kaggle.com/code/mathan/factor-analysis-on-match-statistics

In [38]:
# Imports 

import numpy as np
from sklearn.decomposition import FactorAnalysis
import matplotlib.pyplot as plt
import pandas as pd
from factor_analyzer.factor_analyzer import calculate_kmo

In [39]:
# Load data
dataframe = pd.read_csv('FIFA_2018_Statistics.csv')

In [40]:
# Preprocess
dataframe = dataframe.fillna(0) # Remove NA values
dataframe = dataframe.select_dtypes(include=[np.number]) # Remove string columns
corr = dataframe.corr() # Compute correlation matrix

In [43]:
"""The Kaiser-Meyer-Olkin (KMO) test is a measure of sampling adequacy for factor analysis. 
It is used to determine whether the data is suitable for factor analysis. The KMO value ranges from 0 to 1, with values closer to 1 indicating that the data is more suitable for factor analysis. 
In general, a KMO value above 0.6 is considered acceptable, and a value above 0.8 is considered good."""

# calculate KMO value
kmo_all, kmo_model = calculate_kmo(corr)
 
# print KMO value
variablesToKeep = []
for index, variable in enumerate(corr.columns):
    if kmo_all[index] > 0.6:
        print('KMO value for', variable, ': ', kmo_all[index])
        variablesToKeep.append(variable)
        
# Remove variables with KMO < 0.6
dataframe = dataframe[variablesToKeep]

KMO value for Ball Possession % :  0.6598320585522958
KMO value for On-Target :  0.7968240502156893
KMO value for Blocked :  0.6269368187826782
KMO value for Corners :  0.6501822888455828
KMO value for Pass Accuracy % :  0.7708021966377492
KMO value for Own goals :  0.6623291411373146


c:\Users\laurence\miniconda3\envs\sandbox\lib\site-packages\factor_analyzer\utils.py:244: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn(
